# Single Pass Agent Execution

This notebook runs a complete pass of the Prediction Agent on a single live Kalshi market.

**Steps:**
1. Initialize Kalshi Client & Tool Registry
2. Fetch Active Markets
3. Invoke Agent (AWS Bedrock) to Select Tools
4. Execute Tools & Compute Score
5. Review Decision

In [ ]:
import logging
import sys
from datetime import datetime, timezone
from pathlib import Path
import json

# Ensure project root is in path
# Use '../..' to move from 'prediction_agent/one_pass/' to repo root 'Prediction_Agent/'
PROJECT_ROOT = Path("../..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from prediction_agent.api.kalshi_client import KalshiClient
from prediction_agent.schemas import EventInput, FormulaSpec
from prediction_agent.agent.nodes import select_tools_node
from prediction_agent.engine.tool_runner import run_tools
from prediction_agent.engine.scorer import compute_score
from prediction_agent.tools.registry import build_default_registry

# Configure Logging
logging.basicConfig(level=logging.INFO, format="%(message)s", force=True)
logger = logging.getLogger("notebook")

## 1. Initialize Client & Registry

In [ ]:
try:
    client = KalshiClient()
    print("✅ Kalshi Client Initialized")
except Exception as e:
    print(f"❌ Client Init Failed: {e}")

registry = build_default_registry()
print(f"✅ Tool Registry Built ({len(registry)} tools available)")

## 2. Fetch Active Market

In [ ]:
markets = client.get_active_markets(limit=5)

if not markets:
    print("⚠️ No active markets found.")
else:
    chosen = markets[0]
    print(f"Selected Market:\n  Title: {chosen.get('title')}\n  ID:    {chosen['market_id']}\n  Price: {chosen.get('last_price')}")

    # Create EventInput
    event = EventInput(
        event_id=chosen.get("event_id", chosen["market_id"]),
        market_id=chosen["market_id"],
        market_title=chosen.get("title", "Unknown"),
        current_price=chosen.get("last_price", 0.0),
        timestamp=datetime.now(timezone.utc)
    )

## 3. Agent Analysis (AWS Bedrock)
The agent will review the market and available tools to create a `FormulaSpec`.

In [ ]:
print("🤖 Invoking Agent...")

state = {
    "event_input": event.model_dump(),
    "tools_list": registry.list_tools(),
    "formula_spec": None,
    "error": None
}

# Run the node
state = select_tools_node(state)

if state.get("error"):
    print(f"⚠️ Agent Warning: {state['error']}")

spec_data = state.get("formula_spec")
if spec_data:
    spec = FormulaSpec(**spec_data)
    print(f"\n📝 Rationale: {spec.rationale}")
    print(f"🎯 Threshold: {spec.threshold}")
    print("\n🛠️ Selected Tools:")
    for sel in spec.selections:
        print(f"   - {sel.tool_name} (Weight: {sel.weight:.2f})")
else:
    print("❌ Agent failed to produce a spec.")

## 4. Execute Tools & Score
Run the selected tools to get their numeric signals, then compute the weighted score.
**NOTE:** We automatically append the current snapshot to the dataset so tools see fresh data.

In [ ]:
print("💾 Appending current snapshot to history (for tool freshness)...")
snapshot_entry = {
    "timestamp": event.timestamp.isoformat(),
    "market_id": event.market_id,
    "title": event.market_title,
    "last_price": event.current_price,
    "status": chosen.get("status", "active"),
    "yes_bid": chosen.get("yes_bid", 0.0),
    "yes_ask": chosen.get("yes_ask", 0.0),
    "volume": chosen.get("volume", 0),
    "open_interest": chosen.get("open_interest", 0),
    "liquidity": chosen.get("liquidity", 0.0),
    "close_time": chosen.get("close_time"),
}

snapshots_path = PROJECT_ROOT / "prediction_agent" / "outputs" / "market_snapshots.jsonl"
try:
    with open(snapshots_path, "a", encoding="utf-8") as f:
        f.write(json.dumps(snapshot_entry) + "\n")
    print("✅ Snapshot saved.")
except Exception as e:
    print(f"⚠️ Failed to save snapshot: {e}")

print("\n⚙️ Executing Tools...")
tool_outputs = run_tools(event, spec, registry)

for out in tool_outputs:
    print(f"   -> {out.tool_name}: {out.output_vector}")

print("\n📊 Computing Score...")
score_result = compute_score(tool_outputs, spec)

print(f"\n{'='*40}")
print(f"FINAL SCORE: {score_result.final_score:.4f}")
print(f"THRESHOLD:   {score_result.threshold:.4f}")
print(f"DECISION:    {'✅ BET (YES)' if score_result.bet_triggered else '❌ NO BET'}")
print(f"{'='*40}")